In [75]:
# 해야할 일: 자동 포맷 복사, 어떻게 gs를 아래로 계속 내리면서 기입할 것인가, 최근 데이터를 한 눈에 볼 수 있는 지표 만들기
# 마지막 고려사항: 시간에 맞춰 자동 동작

In [76]:
#af 클릭 자동 모니터링

In [77]:
# !pip install chromedriver-autoinstaller
from selenium.webdriver.support.select import Select
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from fake_useragent import UserAgent
from bs4 import BeautifulSoup
import time
import datetime
from oauth2client.service_account import ServiceAccountCredentials #스프레드 시트 관련
import gspread

In [78]:
# 앱스플라이어 자동로그인

options = Options()
ua = UserAgent()
userAgent = ua.random
print(userAgent)
options.add_argument(f'user-agent={userAgent}')
driver = webdriver.Chrome(chrome_options=options, executable_path=r'./chromedriver.exe')
# driver.get("http://admin.mobiconnect.co.kr/admin/login")
# driver.quit()

Opera/9.80 (Windows NT 5.1; U;) Presto/2.7.62 Version/11.01


<ipython-input-78-ca3ef673975a>:8: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chrome_options=options, executable_path=r'./chromedriver.exe')


In [79]:
today= str(datetime.datetime.now().strftime("%Y-%m-%d"))
today_time = str(datetime.datetime.now().strftime("%H:%M"))
days_ago = datetime.datetime.now() - datetime.timedelta(days=1)
yesterday= str(days_ago.strftime("%Y-%m-%d"))
today
today_time

'18:36'

In [80]:
day_name = ['월','화','수','목','금','토','일']
b = day_name[datetime.datetime.today().weekday()]
print(b)

목


In [81]:
today_hum = str(today)+"("+str(b)+") "+str(today_time)
today_hum #구글 스프레드 시트에 기입할 날짜 + 요일 + 시간`

'2021-06-24(목) 18:36'

In [82]:
time1 = datetime.datetime(2021, 6, 23, 9, 0, 0) #기준일 21/06/23
time2 = datetime.datetime.now()
date_gs = (time2-time1).days # 기준날짜와 오늘 날짜의 차
date_gs

1

In [83]:
    # 구글 시트에 연결(for문으로 돌릴 필요가 없어 위로 옮김)
scope = ['https://spreadsheets.google.com/feeds']
json_file_name = './rpadaily-a0b4f0f12ff6.json'
credentials = ServiceAccountCredentials.from_json_keyfile_name(json_file_name, scope)
gc = gspread.authorize(credentials)
#찾아갈 시트주소
spreadsheet_url = 'https://docs.google.com/spreadsheets/d/15r6Ts8XR_rZ90wZ--gcgrdXwu43tydEN6MWyCmHPukg/edit#gid=1687502285'
    
doc = gc.open_by_url(spreadsheet_url)
sheet1 = doc.worksheet('AF모니터링의 사본')

# 현재 시간 기입 (수정 필요)
sheet1.update_acell('a'+str(8+date_gs), today_hum)

{'spreadsheetId': '15r6Ts8XR_rZ90wZ--gcgrdXwu43tydEN6MWyCmHPukg',
 'updatedRange': "'AF모니터링의 사본'!A9",
 'updatedRows': 1,
 'updatedColumns': 1,
 'updatedCells': 1}

In [84]:
#캠페인 추가시 수정할 부분(미디어 계정은 _media)

campaign_dic = {'realfarm':"kr.neogames.realfarm", 'godticket':"godticket.mobile", 'grow':"kr.co.grow.app", 'goldspoon_aos':"com.goldspoon", 'goldspoon_ios':"id1332604123", 'lord_kr_aos': "com.clovergames.lordofheroes", 'lord_us_aos': "com.clovergames.lordofheroes", 'lord_kr_ios':"id1496742556", 'lord_us_ios':"id1496742556", "vanila_media":"kr.projectvanilla.production"}
# ios 포함으로 변경 필요
campaign_list = list(campaign_dic.keys())
campaign_list

['realfarm',
 'godticket',
 'grow',
 'goldspoon_aos',
 'goldspoon_ios',
 'lord_kr_aos',
 'lord_us_aos',
 'lord_kr_ios',
 'lord_us_ios',
 'vanila_media']

In [85]:
# 아래에서 사용(스프레드 시트 열지정)
#캠페인 추가시 수정할 부분(미디어 계정은 _media)
campaign_gsnum_dic = {'realfarm':"g", 'godticket':"k", 'grow':"j", 'goldspoon_aos':"h", 'goldspoon_ios':"i", 'lord_kr_aos':"c", 'lord_kr_ios':"d", 'lord_us_aos':"e", 'lord_us_ios':"f", "vanila_media":"l" }

In [86]:
# #임시
# driver.get('https://hq1.appsflyer.com/dashboard/overview/kr.neogames.realfarm#end=2021-06-23&grouping=attribution&start=2021-06-23')
# driver.find_element_by_id('user-email').send_keys('globalsales@mobidays.com')
# driver.find_element_by_id('password-field').send_keys('Mobi2020#@!$')
# driver.find_element_by_xpath('//*[@id="login-form"]/div[6]/button').click()

In [88]:
cnt=0
# campaign_aos는 캠페인 명
for campaign in campaign_list:
# 앱스플라이어 로그인
    driver.get('https://hq1.appsflyer.com/dashboard/overview/'+campaign_dic.get(campaign)+'#end='+today+'&grouping=attribution&mediaSource=mobione_int&start='+today)
    
    time.sleep(0.4)

    if campaign[-6:]!="_media":    
        try: #로그인 필요 없으면 패스
            driver.find_element_by_id('user-email').send_keys('globalsales@mobidays.com')
            driver.find_element_by_id('password-field').send_keys('Mobi2020#@!$')
            driver.find_element_by_xpath('//*[@id="login-form"]/div[6]/button').click()
        except:
            pass
        time.sleep(6.5)
        
        if campaign in ["lord_kr_aos", "lord_kr_ios"]:
            time.sleep(1.5)
            lord_type = driver.find_element_by_css_selector('div.webix_ss_body > div.webix_ss_left > div > div > div:nth-child(1)').text
            if lord_type[-2:] =="KR":
                click= driver.find_element_by_css_selector('div.webix_ss_body > div.webix_ss_center > div > div:nth-child(3) > div:nth-child(1)').text
                install = driver.find_element_by_css_selector('div.webix_ss_body > div.webix_ss_center > div > div.webix_column.af-webix-table-integer-column.af-webix-table-thin-grouped-column.af-webix-start-table-section-column > div:nth-child(1)').text
            elif lord_type[-2:] =="US": #첫번째 캠페인이 us면 두번째 캠페인 정보를 kr로 가져오겠다
                click= driver.find_element_by_css_selector('div.webix_ss_body > div.webix_ss_center > div > div:nth-child(3) > div:nth-child(2)').text
                install = driver.find_element_by_css_selector('div.webix_ss_body > div.webix_ss_center > div > div.webix_column.af-webix-table-integer-column.af-webix-table-thin-grouped-column.af-webix-start-table-section-column > div:nth-child(2)').text
            else:
                click="국가명 확인필요"
                install="국가명 확인필요"
        
    
        elif campaign in ["lord_us_aos", "lord_us_ios"]:
            time.sleep(1.5)
            lord_type = driver.find_element_by_css_selector('div.webix_ss_body > div.webix_ss_left > div > div > div:nth-child(1)').text
            if lord_type[-2:] =="US": #첫번째 캠페인이 US면 첫번째 테이블의 정보를 US로 가져오겠다
                click= driver.find_element_by_css_selector('div.webix_ss_body > div.webix_ss_center > div > div:nth-child(3) > div:nth-child(1)').text
                install = driver.find_element_by_css_selector('div.webix_ss_body > div.webix_ss_center > div > div.webix_column.af-webix-table-integer-column.af-webix-table-thin-grouped-column.af-webix-start-table-section-column > div:nth-child(1)').text
            elif lord_type[-2:] =="KR":
                click= driver.find_element_by_css_selector('div.webix_ss_body > div.webix_ss_center > div > div:nth-child(3) > div:nth-child(2)').text
                install = driver.find_element_by_css_selector('div.webix_ss_body > div.webix_ss_center > div > div.webix_column.af-webix-table-integer-column.af-webix-table-thin-grouped-column.af-webix-start-table-section-column > div:nth-child(2)').text
            else:
                click="국가명 확인필요"
                install="국가명 확인필요"


        else:       #로드오브 히어로즈 외 캠페인들
            time.sleep(0.5)
            click = driver.find_element_by_css_selector('div.webix_ss_body > div.webix_ss_center > div > div:nth-child(3)').text
            click
            install = driver.find_element_by_css_selector('div.webix_ss_body > div.webix_ss_center > div > div.webix_column.af-webix-table-integer-column.af-webix-table-thin-grouped-column.af-webix-start-table-section-column > div').text
            install
        
    else:   # 미디어 계정인 경우
        driver.quit()
        driver.get('https://hq1.appsflyer.com/dashboard/overview/'+campaign_dic.get(campaign)+'#end='+today+'&grouping=attribution&mediaSource=mobione_int&start='+today)
        time.sleep(0.5)
    # 앱스플라이어 자동로그인
        options = Options()
        ua = UserAgent()
        userAgent = ua.random
        print(userAgent)
        options.add_argument(f'user-agent={userAgent}')
        driver = webdriver.Chrome(chrome_options=options, executable_path=r'./chromedriver.exe')
        
        driver.find_element_by_id('user-email').send_keys('mediasolution@mobidays.com')
        driver.find_element_by_id('password-field').send_keys('Af010203!dX')
        driver.find_element_by_xpath('//*[@id="login-form"]/div[6]/button').click()
        time.sleep(6)  
        
        click = driver.find_element_by_css_selector('div.webix_ss_body > div.webix_ss_center > div > div:nth-child(3)').text
        click
        install = driver.find_element_by_css_selector('div.webix_ss_body > div.webix_ss_center > div > div.webix_column.af-webix-table-integer-column.af-webix-table-thin-grouped-column.af-webix-start-table-section-column > div').text
        install        
    
    # for문의 campaign을 못받아와서 무조건 실행 while문 사용
    while True:
    # 시간에 따른 셀이동 수정 필요
        sheet1.update_acell(str(campaign_gsnum_dic.get(campaign))+str(8), click)
        sheet1.update_acell(str(campaign_gsnum_dic.get(campaign))+str(9), install)
        break;


    
    
# driver.quit()

MaxRetryError: HTTPConnectionPool(host='127.0.0.1', port=49375): Max retries exceeded with url: /session/ad7f793080fd4be125323162f0452d08/url (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x00000260564DEB50>: Failed to establish a new connection: [WinError 10061] 대상 컴퓨터에서 연결을 거부했으므로 연결하지 못했습니다'))